In [1]:
import kagglehub
paultimothymooney_breast_histopathology_images_path = kagglehub.dataset_download('paultimothymooney/breast-histopathology-images')

print('Data import complete.')


100%|██████████| 3.10G/3.10G [00:25<00:00, 129MB/s]

Extracting files...


Data source import complete.


In [2]:
import os
import pandas as pd
import glob
import numpy as np

In [3]:
images = glob.glob('/root/.cache/kagglehub/datasets/paultimothymooney/breast-histopathology-images/versions/1/IDC_regular_ps50_idx5/**/*.png', recursive = True)
images[:5]

['/root/.cache/kagglehub/datasets/paultimothymooney/breast-histopathology-images/versions/1/IDC_regular_ps50_idx5/12897/1/12897_idx5_x2001_y951_class1.png',
 '/root/.cache/kagglehub/datasets/paultimothymooney/breast-histopathology-images/versions/1/IDC_regular_ps50_idx5/12897/1/12897_idx5_x1901_y451_class1.png',
 '/root/.cache/kagglehub/datasets/paultimothymooney/breast-histopathology-images/versions/1/IDC_regular_ps50_idx5/12897/1/12897_idx5_x1751_y501_class1.png',
 '/root/.cache/kagglehub/datasets/paultimothymooney/breast-histopathology-images/versions/1/IDC_regular_ps50_idx5/12897/1/12897_idx5_x2151_y1001_class1.png',
 '/root/.cache/kagglehub/datasets/paultimothymooney/breast-histopathology-images/versions/1/IDC_regular_ps50_idx5/12897/1/12897_idx5_x2001_y1101_class1.png']

In [4]:
df = pd.DataFrame(index = np.arange(0,len(images)),columns = ["imgpaths","labels"])

for i in range (0,len(images)):
    df.iloc[i]["imgpaths"] = images[i]
    df.iloc[i]["labels"] = int(images[i][-5])

df.head()

<ipython-input-4-178bfb233677>:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.iloc[i]["imgpaths"] = images[i]
<ipython-input-4-178bfb233677>:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting 

,imgpaths,labels
0,/root/.cache/kagglehub/datasets/paultimothymoo...,1
1,/root/.cache/kagglehub/datasets/paultimothymoo...,1
2,/root/.cache/kagglehub/datasets/paultimothymoo...,1
3,/root/.cache/kagglehub/datasets/paultimothymoo...,1
4,/root/.cache/kagglehub/datasets/paultimothymoo...,1


In [5]:

df.labels=df.labels.map({1:"cancer",0:"no cancer"})

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 277524 entries, 0 to 277523
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   imgpaths  277524 non-null  object
 1   labels    277524 non-null  object
dtypes: object(2)
memory usage: 6.4+ MB


In [8]:
x=df.imgpaths
y=df.labels

In [9]:
y.unique()

array(['cancer', 'no cancer'], dtype=object)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


test_val_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
train_df = pd.DataFrame({'img_path': X_train, 'label': y_train})
val_df = pd.DataFrame({'img_path': X_val, 'label': y_val})
test_df = pd.DataFrame({'img_path': X_test, 'label': y_test})

In [16]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='img_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    color_mode= 'rgb'
)
val_generator = test_val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='img_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)
test_generator = test_val_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='img_path',
    y_col='label',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 194266 validated image filenames belonging to 2 classes.
Found 41629 validated image filenames belonging to 2 classes.
Found 41629 validated image filenames belonging to 2 classes.


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, Adamax

In [18]:
model = Sequential()


model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
model.compile(Adamax(learning_rate= 0.001), loss= 'binary_crossentropy', metrics= ['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 683,329 (2.61 MB)

 Trainable params: 683,329 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.fit(train_generator, epochs= 10, verbose= 1, validation_data= val_generator)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
6071/6071 ━━━━━━━━━━━━━━━━━━━━ 385s 63ms/step - accuracy: 0.8189 - loss: 0.4153 - val_accuracy: 0.8542 - val_loss: 0.3383
Epoch 2/10
6071/6071 ━━━━━━━━━━━━━━━━━━━━ 346s 57ms/step - accuracy: 0.8535 - loss: 0.3482 - val_accuracy: 0.8599 - val_loss: 0.3257
Epoch 3/10
6071/6071 ━━━━━━━━━━━━━━━━━━━━ 333s 55ms/step - accuracy: 0.8601 - loss: 0.3314 - val_accuracy: 0.8639 - val_loss: 0.3217
Epoch 4/10
6071/6071 ━━━━━━━━━━━━━━━━━━━━ 327s 54ms/step - accuracy: 0.8671 - loss: 0.3181 - val_accuracy: 0.8762 - val_loss: 0.2943
Epoch 5/10
6071/6071 ━━━━━━━━━━━━━━━━━━━━ 335s 55ms/step - accuracy: 0.8698 - loss: 0.3119 - val_accuracy: 0.8679 - val_loss: 0.3107
Epoch 6/10
6071/6071 ━━━━━━━━━━━━━━━━━━━━ 327s 54ms/step - accuracy: 0.8694 - loss: 0.3104 - val_accuracy: 0.8783 - val_loss: 0.2965
Epoch 7/10
6071/6071 ━━━━━━━━━━━━━━━━━━━━ 332s 55ms/step - accuracy: 0.8732 - loss: 0.3057 - val_accuracy: 0.8801 - val_loss: 0.2842
Epoch 8/10
6071/6071 ━━━━━━━━━━━━━━━━━━━━ 377s 54ms/step - accuracy: 

In [21]:
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_acc * 100:.2f}%")

1301/1301 ━━━━━━━━━━━━━━━━━━━━ 25s 20ms/step - accuracy: 0.8761 - loss: 0.2880
Test Accuracy: 87.99%


In [28]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='img_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode= 'rgb',
    validate_filenames=True
)
val_generator = test_val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='img_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    validate_filenames=True
)
test_generator = test_val_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='img_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    validate_filenames=True
)

Found 194266 validated image filenames belonging to 2 classes.
Found 41629 validated image filenames belonging to 2 classes.
Found 41629 validated image filenames belonging to 2 classes.


In [29]:

import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import VGG16
# from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
# from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze pretrained layers

x = Flatten()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)  # Dropout to prevent overfitting
x = Dense(2, activation="softmax")(x)  # 2 classes (0 or 1)

model = Model(inputs=base_model.input, outputs=x)


In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model

# Load Pretrained Model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze all layers initially

# Add Custom Layers
x = GlobalAveragePooling2D()(base_model.output)  # Better than Flatten
x = Dense(256, activation="relu")(x)
x = BatchNormalization()(x)  # Improves convergence
x = Dropout(0.5)(x)  # Dropout to prevent overfitting
x = Dense(2, activation="softmax")(x)  # 2 classes (0 or 1)

# Define Model
model = Model(inputs=base_model.input, outputs=x)

# Enable Fine-Tuning (Unfreeze last few layers)
for layer in base_model.layers[-4:]:  # Adjust the number of layers to unfreeze
    layer.trainable = True

# Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Summary
model.summary()


In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
# model.compile(Adamax(learning_rate= 0.001), loss= 'binary_crossentropy', metrics= ['accuracy'])
# model.compile(optimizer="adam", loss= 'binary_crossentropy', metrics= ['accuracy'])


# Train the model

# model.fit(train_generator, epochs= 10, verbose= 1, validation_data= val_generator)
model.fit(
    train_generator,
    # validation_data=test_generator,
    validation_data=val_generator,
    epochs=10,
    verbose= 1,
    steps_per_epoch=len(train_generator),
    validation_steps=len(test_generator)
)

# Evaluate the model

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
6071/6071 ━━━━━━━━━━━━━━━━━━━━ 2627s 430ms/step - accuracy: 0.7933 - loss: 0.4775 - val_accuracy: 0.8281 - val_loss: 0.3950
Epoch 2/10
6071/6071 ━━━━━━━━━━━━━━━━━━━━ 2584s 426ms/step - accuracy: 0.8065 - loss: 0.4223 - val_accuracy: 0.8052 - val_loss: 0.4038
Epoch 3/10
6071/6071 ━━━━━━━━━━━━━━━━━━━━ 2594s 424ms/step - accuracy: 0.8073 - loss: 0.4177 - val_accuracy: 0.8297 - val_loss: 0.3764
Epoch 4/10
5195/6071 ━━━━━━━━━━━━━━━━━━━━ 5:39 388ms/step - accuracy: 0.8113 - loss: 0.4117

In [ ]:

test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_acc * 100:.2f}%")

plt.plot(model.history['accuracy'])
plt.plot(model.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()